# Домашнее задание к лекции «Коллаборативная фильтрация»

In [80]:
from surprise import SVD, KNNBaseline, KNNWithMeans, KNNBasic, KNNWithZScore, SVDpp, SlopeOne, NMF, BaselineOnly, CoClustering
from surprise import Dataset, Reader, accuracy
from surprise.model_selection import cross_validate, GridSearchCV

from surprise.model_selection import train_test_split

import pandas as pd

Загружаем данные, timestamp из сета для обучения убираем

In [87]:
reader = Reader(rating_scale=(1, 5))

ratings = pd.read_csv('ratings.csv')

ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [88]:
data = Dataset.load_from_df(ratings[['userId','movieId','rating']], reader)

In [89]:
def fit_model(models, df, measures=['RMSE'], cv=5, verbose=False):        
    res = []
    
    for model in models:
        results = cross_validate(model, df, measures=measures, cv=cv, verbose=verbose)  

        tmp = pd.DataFrame.from_dict(results).mean(axis=0)
        tmp = tmp.append(pd.Series([str(model).split(' ')[0].split('.')[-1]], index=['Model']))
        res.append(tmp)
    
    print(pd.DataFrame(res).set_index('Model'))

Строим модели

In [90]:
fit_model([SVD(), SVDpp(verbose=False), KNNBaseline(verbose=False), KNNWithMeans(verbose=False), KNNBasic(verbose=False),
           KNNWithZScore(verbose=False), SlopeOne(), NMF(verbose=False), BaselineOnly(verbose=False),
           CoClustering(verbose=False)], data)

               test_rmse    fit_time  test_time
Model                                          
SVD             0.873918    4.923804   0.160996
SVDpp           0.862437  499.681281   7.589753
KNNBaseline     0.874402    0.459399   1.811401
KNNWithMeans    0.897323    0.233406   1.536981
KNNBasic        0.945980    0.205001   1.358593
KNNWithZScore   0.894895    0.298803   1.648589
SlopeOne        0.899447    4.375595   5.203805
NMF             0.920964    5.538606   0.192411
BaselineOnly    0.872702    0.258388   0.080999
CoClustering    0.942561    2.651201   0.083008


Как видно из таблицы, лучший результат получился у модели SVDpp: 0.86, SVD и BaselineOnly тоже очень близко.

В итоге результат меньше 0,87 можно получить и без доп. настройки гиперпараметров, в предыдущем решении высокий rsme был судя по всему из-за того, что я не убрала timestamp из данных